In [2]:
# Import the requests library.
import requests as r
import pandas as pd
import numpy as np
from itertools import product

# Import the API key.
from config import Yelp_API_Key

In [4]:
# Define the API key, Endpoint, and Header
key = Yelp_API_Key
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization':'bearer %s' % key}

In [5]:
# Reading in the zip code csv file that we extracted from the HH Income dataset
zips_data_df = pd.read_csv("data/tiny_zips_for_Melissa.csv", index_col=None,dtype={"Zip_Code":"string"})
zips_data_df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
# Scrape

# Include 20 addresses and 100 results
addresses = zips_data_df
offset = np.arange(0,100,50)
# Create list of tuples of combinations between addresses and offset
tuples = list(product(addresses, offset))

In [7]:
# Use a for loop store results into list of lists, and turn that into a DF

# Empty list
list = []

# Future DF columns
cols = ['Name', 'Type', 'Address', 'Zip Code']

# For loops
for index, row in addresses.iterrows():
    current_zip = row['Zip_Code']
    search_parameters = {
        'location': current_zip,
        'term': 'fast food',
        'radius': 5000,
        'limit': 50,
        'offset': index}
    resp = r.get(url, headers=headers, params=search_parameters)
    raw_data = resp.json()
    print(current_zip)
    
    try:
         for business in raw_data['businesses']:
                name = business['name'],
                food_type = business['categories'][0]['alias']
                address = business['location']['address1'],
                zip_code = business['location']['zip_code'],
                list.append([name, food_type, address, zip_code])
    except KeyError as e:
        print (f"Caught an error at line {e}")
        pass
    
df = pd.DataFrame.from_records(list, index='Name', columns=cols)
print(f'Total Records: {len(df)}')

df.head(20)

01201
01220
01223
01151
01104
01107
01109
01119
01105
01108
01118
01089
01085
01095
01081
01082
01093
01106
01092
01225
Total Records: 253


,Type,Address,Zip Code
Name,,,
"(The Marketplace Café,)",sandwiches,"(53 North St,)","(01201,)"
"(Hot Harry's Fresh Burritos,)",mexican,"(37 North St,)","(01201,)"
"(McDonald's,)",burgers,"(445 W Housatonic St,)","(01201,)"
"(Otto's Kitchen & Comfort,)",breakfast_brunch,"(95 East St,)","(01201,)"
"(Burger King,)",burgers,"(29 1st St,)","(01201,)"
"(The Lantern Bar & Grill,)",newamerican,"(455 North St,)","(01201,)"
"(Empire Pizza Restaurant,)",pizza,"(413 North St,)","(01201,)"
"(Panchos Mexican Restaurant,)",mexican,"(154 North St,)","(01201,)"
"(Ginos Pizza and Grill,)",pizza,"(1206 North St,)","(01201,)"


In [11]:
df.to_csv('data/My_20_Zips_Results.csv', columns = ["Type", "Address", "Zip Code"], index_label= "Name", header = True)